# Introduction
Density-Based Spatial Clustering of Applications with Noise (DBSCAN), is a data clustering algorithm that works by identifying clusters of high density points separated by regions of low density. Unlike K-Means or GMMs that rely on distances or predefined centres, DBSCAN focuses on the density of points to group them.